# <div align="center"> 目标探测 </div>

In [2]:
from IPython.display import Markdown as md
import requests
import json
import consul
import time
import socket

## 1 全局配置

### 1.1 函数

In [3]:
# 获取本机器IP
def get_host_ip():
    ip=''
    try:
        s = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
        s.connect(('8.8.8.8',80))
        ip = s.getsockname()[0]
    finally:
        s.close()
    return ip

def delete_datacenter_key(key):
    try:
        client = consul.Consul(consul_addr, consul_port)
        client.kv.delete(key)
    except:
        pass

# API异步接口将过程中产出的数据同步到consul数据中心, 调用该接口可从中心获取数据 (数据中心仅供开发使用)
def query_async_data(key, trycnt=3):
    i = 0
    while i < trycnt:
        try:
            client = consul.Consul(consul_addr, consul_port)
            _, data = client.kv.get(key)
            return str(data['Value'], encoding="utf-8")
        except:
            i = i + 1
            time.sleep(2)
    return 'Query nothing!!!'
    
# 格式化打印json字符串
def print_json(text):
    if isinstance(text, str):
        print(json.dumps(json.loads(text), indent=4))
    else:
        print(json.dumps(text, indent=4))

### 1.2 变量

In [4]:
host = get_host_ip()
port = 8119

user = 'test'
uuid = 'abcdef'

consul_addr = !echo $(curl -s http://txt.go.sohu.com/ip/soip)| grep -P -o -i "(\d+\.\d+.\d+.\d+)"
consul_port = 8500
consul_addr = consul_addr[0]

query_trycnt = 2
max_epoch = 2

### 1.3 全局配置

In [5]:
det_template_config = '''{
    "dataset": "voc",
    "task": "det",
    "method": "single_shot_detector",
    "data": {
      "num_classes": 21,
      "data_dir": "~/DataSet/VOC07+12_DET",
      "image_tool": "cv2",
      "input_mode": "BGR",
      "keep_difficult": false,
      "workers": 1,
      "mean_value": [104, 117, 123],
      "normalize": {
        "div_value": 1,
        "mean": [104.0, 117.0, 123.0],
        "std": [1.0, 1.0, 1.0]
      }
    },
    "train": {
      "batch_size": 16,
      "aug_trans": {
        "shuffle_trans_seq": ["random_contrast", "random_hue", "random_saturation", "random_brightness", "random_perm"],
        "trans_seq": ["random_hflip", "random_pad", "random_det_crop"],
        "random_saturation": {
          "ratio": 0.5,
          "lower": 0.5,
          "upper": 1.5
        },
        "random_hue": {
          "ratio": 0.5,
          "delta": 18
        },
        "random_contrast": {
          "ratio": 0.5,
          "lower": 0.5,
          "upper": 1.5
        },
        "random_pad": {
          "ratio": 0.6,
          "up_scale_range": [1.0, 4.0]
        },
        "random_brightness": {
          "ratio": 0.5,
          "shift_value": 32
        },
        "random_perm": {
          "ratio": 0.5
        },
        "random_hflip": {
          "ratio": 0.5,
          "swap_pair": []
        },
        "random_det_crop":{
          "ratio": 1.0
        }
      },
      "data_transformer": {
        "size_mode": "fix_size",
        "input_size": [300, 300],
        "align_method": "only_scale"
      }
    },
    "val": {
      "batch_size": 16,
      "use_07_metric": true,
      "aug_trans": {
        "trans_seq": []
      },
      "data_transformer": {
        "size_mode": "fix_size",
        "input_size": [300, 300],
        "align_method": "only_scale"
      }
    },
    "test": {
      "batch_size": 16,
      "aug_trans": {
        "trans_seq": []
      },
      "data_transformer": {
        "size_mode": "fix_size",
        "input_size": [300, 300],
        "align_method": "only_scale"
      }
    },
    "anchor": {
      "anchor_method": "ssd",
      "iou_threshold": 0.5,
      "num_anchor_list": [4, 6, 6, 6, 4, 4],
      "feature_maps_wh": [[38, 38], [19, 19], [10, 10], [5, 5], [3, 3], [1, 1]],
      "cur_anchor_sizes": [30, 60, 111, 162, 213, 264, 315],
      "aspect_ratio_list": [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
    },
    "details": {
      "color_list": [[255, 170, 30], [0, 0, 70], [244, 35, 232]],
      "name_id_dict": {
        "aeroplane": 1, "bicycle": 2, "bird": 3, "boat": 4, "bottle": 5, "bus": 6, "car": 7,
        "cat": 8, "chair": 9, "cow": 10, "diningtable": 11, "dog": 12, "horse": 13, "motorbike": 14,
        "person": 15, "pottedplant": 16, "sheep": 17, "sofa": 18, "train": 19, "tvmonitor": 20
      },
      "name_seq": ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair",
                   "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant",
                   "sheep", "sofa", "train", "tvmonitor"]
    },
    "network":{
      "backbone": "vgg16",
      "model_name": "vgg300_ssd",
      "num_feature_list": [512, 1024, 512, 256, 256, 256],
      "stride_list": [8, 16, 30, 60, 100, 300],
      "head_index_list": [0, 1, 2, 3, 4, 5],
      "checkpoints_name": "ssd_vgg300_voc_det",
      "checkpoints_dir": "checkpoints"
    },
    "solver": {
      "lr": {
        "metric": "epoch",
        "is_warm": true,
        "warm": {
          "warm_iters": 1000,
          "power": 1.0,
          "freeze_backbone": false
        },
        "base_lr": 0.001,
        "lr_policy": "multistep",
        "multistep": {
          "gamma": 0.1,
          "stepvalue": [156, 195, 234]
        }
      },
      "optim": {
        "optim_method": "sgd",
        "adam": {
          "betas": [0.9, 0.999],
          "eps": 1e-08,
          "weight_decay": 0.0001
        },
        "sgd":{
          "weight_decay": 0.0005,
          "momentum": 0.9,
          "nesterov": false
        }
      },
      "display_iter": 100,
      "save_iters": 5000,
      "test_interval": 5000,
      "max_epoch": 2
    },
    "res": {
      "nms": {
        "mode": "union",
        "max_threshold": 0.45,
        "pre_nms": 1000
      },
      "val_conf_thre": 0.01,
      "vis_conf_thre": 0.5,
      "max_per_image": 200,
      "cls_keep_num": 50
    },
    "loss": {
      "loss_type": "multibox_loss",
      "loss_weights": {
        "multibox_loss": {
            "multibox_loss": 1.0
        }
      }
    }
}'''
det_template_config_dict = json.loads(det_template_config)

## 2 训练

In [6]:
det_config = det_template_config_dict

det_config['dataset'] = 'voc'
det_config['data']['data_dir'] = '/data/datasets/cv/VOC07+12_DET'
det_config['network']['model_name'] = 'vgg16_ssd300'

conf = json.dumps(det_config)

### train.stop

In [8]:
op = 'train.stop'
uuid = 'vgg300_ssd_voc'
data = '''{
    "op":"%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": {}
}''' % (op, user, uuid)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

{
    "code": 100200,
    "descr": "task service success"
}


### train.start

In [10]:
op = 'train.start'
uuid = 'vgg300_ssd_voc'
data = '''{
    "op":"%s",
    "user": "%s",
    "service_name": "k12cv",
    "service_uuid": "%s",
    "service_params": %s
}''' % (op, user, uuid, conf)

api = 'http://%s:%d/k12ai/framework/train' % (host, port)
print_json(requests.post(url=api, json=json.loads(data)).text)

key = 'framework/%s/%s/%s/metrics'%(user, uuid, op)
delete_datacenter_key(key)

{
    "code": 100200,
    "descr": "task service success"
}


输出样例:
```json
{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "vgg300_ssd_voc",
    "timestamp": 1575045096684,
    "datetime": "2019-11-30 00:31:36",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 14.5297184,
        "training_speed": 0.603,
        "lr": [
            0.0002,
            0.001
        ]
    }
}
```

In [11]:
print(query_async_data(key, query_trycnt))

{
    "version": "0.1.0",
    "type": "metrics",
    "tag": "framework",
    "op": "train.start",
    "user": "test",
    "service_uuid": "vgg300_ssd_voc",
    "timestamp": 1575382955624,
    "datetime": "2019-12-03 22:22:35",
    "metrics": {
        "training_epochs": 1,
        "training_loss": 8.15280151,
        "training_speed": 0.553,
        "lr": [
            0.0007,
            0.001
        ]
    }
}


## 3 评估